# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 03: Online Inference</span>

<span style="font-width:bold; font-size: 1.4rem;">In this last notebook you will use your deployment for online inference.  </span>

## **🗒️ This notebook is divided into the following sections:** 
1. **Deployment Retrieval**: Retrieve your deployment from the model registry.
2. **Prediction using deployment**.
3. **REST endpoint usage for model serving**.

![tutorial-flow](../images/03_model.png)

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [2]:
import hopsworks

project = hopsworks.login()

# Get the feature store handle for the project's feature store
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://staging.cloud.hopsworks.ai/p/120
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;">🗄 Model Registry</span>


In [3]:
# Get the Model Registry
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


## <span style='color:#ff5f27'>⚙️ Fetch Deployment</span>

In [ ]:
# Access the Model Serving
ms = project.get_model_serving()

# Specify the deployment name
deployment_name = "amlmodeldeployment"

# Get the deployment with the specified name
deployment = ms.get_deployment(deployment_name)

# Start the deployment and wait for it to be in a running state for up to 300 seconds
deployment.start(await_running=300)

## <span style='color:#ff5f27'>🔮 Predicting using deployment</span>


Finally you can start making predictions with your model!

Send inference requests to the deployed model as follows:

In [ ]:
# Prepare input data using the input example from the model
data = {
    "inputs": model.input_example,
}

# Make predictions using the deployed model
predictions = deployment.predict(data)

In [ ]:
# Prepare input data with a specific input value
data = {
    "inputs": ["f1c119ed"],
}

# Make predictions using the deployed model
predictions = deployment.predict(data)

> For trouble shooting one can use `get_logs` method.

In [ ]:
deployment.get_logs()

## <span style='color:#ff5f27'>🚀 Use REST endpoint</span>

You can also use a REST endpoint for your model. To do this you need to create an API key with 'serving' enabled, and retrieve the endpoint URL from the Model Serving UI.

Go to the Model Serving UI and click on the eye icon next to a model to retrieve the endpoint URL. The shorter URL is an internal endpoint that you can only reach from within Hopsworks. If you want to call it from outside, you need one of the longer URLs. 

![serving-endpoints](images/serving_endpoints.gif)

In [ ]:
import os
import requests

mr = project.get_model_registry()

# Use the model name from the previous notebook.
model = mr.get_model(
    name="fraud_tutorial_model", 
    version=1,
)

test_inputs = model.input_example

API_KEY = "..."  # Put your API key here.
MODEL_SERVING_URL = "..." # Put model serving endppoint here.
HOST_NAME = "..." # Put your hopsworks model serving hostname here 

data = {"inputs": test_inputs}
headers = {
    "Content-Type": "application/json", "Accept": "application/json",
    "Authorization": f"ApiKey {API_KEY}",
    "Host": HOST_NAME}

response = requests.post(MODEL_SERVING_URL, verify=False, headers=headers, json=data)
response.json()

In [ ]:
# Now lets test feature vectors from online store
ids_to_score = [
    "0016359b", 
    "001dcc27", 
    "0054a022", 
    "00d6b609", 
    "00e14860", 
    "00e39a1b", 
    "014ed5cb", 
    "01ce3306", 
    "01fa19ae", 
    "01fa1d01", 
    "036dce03", 
    "03e09be4", 
    "04b23f4b",
]

for node_id in ids_to_score:
    data = {"inputs": [node_id]}
    print(" anomaly score for node_id ", node_id, " : ",   deployment.predict(data)["outputs"])

## Stop Deployment
To stop the deployment you simply run:

In [ ]:
deployment.stop()

## <span style="color:#ff5f27;"> 🎁 Wrapping things up </span>

In this module you perforemed feature engineering, created feature view and traning dataset, trained advesarial anomaly detection model and depoyed it in production. To setup this pipeline in your enterprise settings contuct us.

<img src="images/contuct_us.png" width="400px"></img>